**Imports**

In [481]:
import itertools

import numpy as np

import tqdm

import math

In [2]:
import os

os.chdir('..')

In [3]:
from importlib.machinery import SourceFileLoader

eq = SourceFileLoader("eq", "src/equilibria.py").load_module()

from eq import *

In [4]:
from importlib.machinery import SourceFileLoader

main = SourceFileLoader("main", "src/main.py").load_module()

from main import *

In [5]:
import axelrod as axl

In [6]:
from axelrod.action import Action

C, D = Action.C, Action.D

**Get memory-one strategy from memory-two**

In [471]:
states = ['CC', 'CD', 'DC', 'DD']

states_in_two_bits = ["".join(i[-2:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

In [472]:
player = [1, 1, 1, 1]

mem_two_player = player * 4

i = 2
coplayer = set_of_memory_one_s[i]

mem_two_coplayer = coplayer * 4

In [496]:
p, c1, c2 = simulate_play(mem_two_player, [0, 0, 0, 0] * 4)

In [497]:
c1

{(C, C): 2, (C, D): 0, (D, C): 47, (D, D): 0}

In [498]:
c2

{(C, C): 1, (C, D): 0, (D, C): 0, (D, D): 0}

In [475]:
25 / (10 * 5), 24 / (10 * 5)

(0.5, 0.48)

In [476]:
ss = calculate_ss_for_mem_two(mem_two_player, mem_two_coplayer)

In [457]:
ss

array([0. , 0.5, 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. ])

In [459]:
ss = [sum([ss[i]  for i in range(16) if states_in_two_bits[i] == state])
            for state in states]

In [477]:
c = [sum([mem_two_coplayer[i]  for i in range(16) if states_in_two_bits[i] == state])
            for state in states]

ModuleNotFoundError: No module named 'maths'

In [460]:
[ss[0], ss[2], ss[1], ss[-1]]

[0.5, 0.0, 0.5, 0.0]

In [461]:
c2

{(C, C): 1, (C, D): 0, (D, C): 24, (D, D): 0}

In [462]:
coplayer

(0, 0, 1, 0)

In [442]:
calculate_ss_for_mem_two(mem_two_player, mem_two_coplayer)

array([0. , 0.5, 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. ])

In [443]:
ss = calculate_ss_for_mem_one(player, coplayer)

In [444]:
np.array([ss[0], ss[2], ss[1], ss[-1]]) * np.array(coplayer) /  (1 / 2)

array([0., 0., 1., 0.])

In [445]:
ss2 = calculate_ss_for_mem_two(mem_two_player, mem_two_coplayer)

In [446]:
ss2 = [sum([ss2[i]  for i in range(16) if states_in_two_bits[i] == state])
            for state in states]

In [464]:
strategy = []
states = [(C, C), (C, D), (D, C), (D, D)]

for state in states:
    if c1[state] > 0:
        strategy.append(c2[state] / c1[state])
    else:
        strategy.append(0)

In [465]:
strategy

[0.04, 0, 1.0, 0]

In [466]:
c2

{(C, C): 1, (C, D): 0, (D, C): 24, (D, D): 0}

In [469]:
c1

{(C, C): 25, (C, D): 0, (D, C): 24, (D, D): 0}

In [470]:
1 / .5

2.0

In [447]:
np.array([ss2[0], ss2[2], ss2[1], ss2[-1]]) * np.array(coplayer) /  (1 / 2)

array([0., 0., 1., 0.])

In [448]:
coplayer

(0, 0, 1, 0)

In [7]:
states = ['CC', 'CD', 'DC', 'DD']

In [8]:
set_of_memory_one_s = list(itertools.product([0, 1], repeat=4))

In [9]:
set_of_memory_two_s = list(itertools.product([0, 1], repeat=16))

In [482]:
def expected_memory_one_strategy(player, coplayer):
    """
    Returns the expected memory-one strategy for a memory-two
    strategy from the player's prespective.
    
    This uses the method by Press and Dyson.
    """
    states = ['CC', 'CD', 'DC', 'DD']
    
    states_in_two_bits = ["".join(i[-2:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

    ss = calculate_ss_for_mem_two(player, coplayer)
    
    ss *= ([coplayer[0], coplayer[2], coplayer[1], coplayer[-1]] * 4)
    
    
#     strategy = [sum([ss[i]  for i in range(16) if states_in_two_bits[i] == state])
#             for state in states]
    
#     return (strategy[0], strategy[1], strategy[2], strategy[-1])

    return [math.ceil(sum([ss[i]  for i in range(16) if states_in_two_bits[i] == state]))
            for state in states]
    

In [483]:
def calculate_ss_for_mem_one(player, coplayer):
    """
    Invariant distribution for memory-one.
    """
    M = calculate_M(player, coplayer)
    ss = invariant_distribution(M)
    
    return ss

def calculate_ss_for_mem_two(player, coplayer):
    """
    Invariant distribution for memory-two.
    """
    M = calculate_M_memory_two(player, coplayer)
    ss = invariant_distribution(M)
    
    return ss

In [499]:
def simulate_play(p, q, turns=10 * 5, noise=0):
    
    states_to_actions = {(C, C): 0, 
                     (C, D): 0, 
                     (D, C): 0, 
                     (D, D):0}

    states_counts = {(C, C): 0, 
                     (C, D): 0, 
                     (D, C): 0, 
                     (D, D):0}

    player = axl.MemoryTwoPlayer([p[0], p[1], p[4], p[5],
                                  p[2], p[3], p[6], p[7],
                                  p[8], p[9], p[12], p[13], 
                                  p[10], p[11], p[14], p[15]])

    coplayer = axl.MemoryTwoPlayer([q[0], q[1], q[4], q[5],
                                    q[2], q[3], q[6], q[7],
                                    q[8], q[9], q[12], q[13], 
                                    q[10], q[11], q[14], q[15]])
    
    
    match = axl.Match(players=[player, coplayer], turns=turns, noise=noise)
    
    _ = match.play()
    
    for i, history in enumerate(match.result[1:-1]):

        states_counts[history[::-1]] += 1
        action = match.result[i + 1][-1]

        if action == C:
            states_to_actions[history[::-1]] += 1
        
        
    strategy = []
    states = [(C, C), (C, D), (D, C), (D, D)]

    for state in states:
        if states_counts[state] > 0:
            strategy.append(states_to_actions[state] / states_counts[state])
        else:
            strategy.append(0)
    
    return strategy, states_counts, states_to_actions

### Sanity check

A memory-one strategy written as a memory two strategy is still the same.

In [485]:
b = 2

c = 1

np.random.seed(0)

for _ in range(1000):
    
    p1, p2, p3, p4 = np.random.random(4)
    
    q1, q2, q3, q4 = np.random.random(4)
    
    ss_one = calculate_ss_for_mem_one([p1, p2, p3, p4], [q1, q2, q3, q4])
    
    ss_two = calculate_ss_for_mem_two([p1, p2, p3, p4] * 4, [q1, q2, q3, q4] * 4)
    
    assert np.isclose(
        ss_one @ np.array([b - c, -c, b, 0]), ss_two @ np.array([b - c, -c, b, 0] * 4) 
    )

Now we can loop over all the possible memory-one strategies, expressed as memory-two strategies, and compare the results. Thus, verify Press and Dyson.

In [486]:
player = [1, 1, 1, 1]

coplayer = list(np.random.random(4))#set_of_memory_one_s[2]

coplayer

[0.45775969986338594,
 0.37691770173244865,
 0.7023351294557564,
 0.20732410502995158]

In [487]:
estimated = expected_memory_one_strategy(player * 4, coplayer * 4)

In [488]:
estimated

[1, 1, 0, 0]

In [363]:
calculate_ss_for_mem_one(player, estimated)

array([0., 1., 0., 0.])

In [359]:
estimated_p, _, _ = simulate_play(player * 4, coplayer * 4)

calculate_ss_for_mem_one(player, estimated_p)

array([0.91836735, 0.08163265, 0.        , 0.        ])

In [360]:
calculate_ss_for_mem_one(player, coplayer)

array([0.94865496, 0.05134504, 0.        , 0.        ])

In [492]:
set_of_memory_one_s[0]

(0, 0, 0, 0)

In [491]:
b = 2

c = 1
# for player in tqdm.tqdm_notebook(set_of_memory_one_s):

player = [1, 1, 1, 1]

for i, coplayer in enumerate(set_of_memory_one_s):

    mem_one_representation = expected_memory_one_strategy(player * 4, coplayer * 4)
    
    simulated_mem_one, _, _ = simulate_play(player * 4, coplayer * 4)

#     ss_estimated = calculate_ss_for_mem_one(player, mem_one_representation)

#     ss = calculate_ss_for_mem_one(player, coplayer)

#     assert (ss == ss_estimated).all()
    print(mem_one_representation, simulated_mem_one)

[0, 0, 0, 0] [0.5, 0, 0.0, 0]
[0, 0, 0, 0] [0.5, 0, 0.0, 0]
[0, 1, 0, 0] [0.04, 0, 1.0, 0]
[0, 1, 0, 0] [0.04, 0, 1.0, 0]
[0, 0, 0, 0] [0.5, 0, 0.0, 0]
[0, 0, 0, 0] [0.5, 0, 0.0, 0]
[0, 1, 0, 0] [0.04, 0, 1.0, 0]
[0, 1, 0, 0] [0.04, 0, 1.0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]
[1, 0, 0, 0] [1.0, 0, 0, 0]


In [85]:
b = 2

c = 1

np.random.seed(1)

random_strategies = [list(np.random.random(4)) for _ in range(3)]

player = [0, 0, 0, 0]

for i, coplayer in enumerate(random_strategies):


    mem_one_representation = expected_memory_one_strategy(player * 4, coplayer * 4)

    ss_estimated = calculate_ss_for_mem_one(player, mem_one_representation)

    ss = calculate_ss_for_mem_one(player, coplayer)

    print(ss, ss_estimated)

[0.         0.         0.51946456 0.48053544] [-0.         -0.          0.18840851  0.81159149]
[-0.         -0.          0.27573781  0.72426219] [-0.         -0.          0.20433821  0.79566179]
[0.         0.         0.59771271 0.40228729] [-0.         -0.          0.28906861  0.71093139]


### Simulations

In [91]:
strategy = [0, 0, 0, 0]

opponent = [1, 0, 1, 0]

In [494]:
b = 2

c = 1


player = [1, 1, 1, 1]

for i, coplayer in enumerate(set_of_memory_one_s):

    mem_one_representation, _, _ = simulate_play(player * 4, coplayer * 4, turns=1000)

    ss_estimated = calculate_ss_for_mem_one(player, coplayer)

    ss = calculate_ss_for_mem_one(player, mem_one_representation)
    
    print(ss, ss_estimated)

    assert (np.isclose(ss,
                       ss_estimated, atol=10**-3).all()), f"{ss} and {ss_estimated}. player: {player} coplayer:{coplayer}"

[0. 1. 0. 0.] [0. 1. 0. 0.]
[0. 1. 0. 0.] [0. 1. 0. 0.]
[0.5005005 0.4994995 0.        0.       ] [0.5 0.5 0.  0. ]
[0.5005005 0.4994995 0.        0.       ] [0.5 0.5 0.  0. ]
[0. 1. 0. 0.] [0. 1. 0. 0.]
[0. 1. 0. 0.] [0. 1. 0. 0.]
[0.5005005 0.4994995 0.        0.       ] [0.5 0.5 0.  0. ]
[0.5005005 0.4994995 0.        0.       ] [0.5 0.5 0.  0. ]
[1. 0. 0. 0.] [1. 0. 0. 0.]
[1. 0. 0. 0.] [1. 0. 0. 0.]
[1. 0. 0. 0.] [1. 0. 0. 0.]
[1. 0. 0. 0.] [1. 0. 0. 0.]
[1. 0. 0. 0.] [1. 0. 0. 0.]
[1. 0. 0. 0.] [1. 0. 0. 0.]
[1. 0. 0. 0.] [1. 0. 0. 0.]
[1. 0. 0. 0.] [1. 0. 0. 0.]


In [158]:
b = 2

c = 1

for player in tqdm.tqdm_notebook(set_of_memory_one_s):

# player = [1, 1, 1, 1]

    for i, coplayer in enumerate(set_of_memory_one_s):

        mem_one_representation, _, _ = simulate_play(player * 4, coplayer * 4, turns=1000)

        ss_estimated = calculate_ss_for_mem_one(player, coplayer)

        ss = calculate_ss_for_mem_one(player, mem_one_representation)

        assert (np.isclose(ss, ss_estimated, atol=10**-3).all()), f"{ss} and {ss_estimated}. player: {player} coplayer:{coplayer}"

/Users/glynatsi/opt/anaconda3/envs/axelrod/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


AssertionError: [0.  0.5 0.  0.5] and [0. 0. 1. 0.]. player: (0, 0, 0, 1) coplayer:(1, 1, 0, 1)